In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("../data/data.csv", encoding='latin1')

In [108]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
535699,581217,22735,RIBBON REEL SOCKS AND MITTENS,1,12/8/2011 9:20,3.29,NaN,United Kingdom
13058,537418,21508,VINTAGE KID DOLLY CARD,12,12/6/2010 15:13,0.42,17884.0,United Kingdom
487658,577774,23351,ROLL WRAP 50'S CHRISTMAS,3,11/21/2011 15:57,1.25,16712.0,United Kingdom
496099,578344,22282,12 EGG HOUSE PAINTED WOOD,1,11/24/2011 9:21,24.96,NaN,United Kingdom
402652,571508,21509,COWBOYS AND INDIANS BIRTHDAY CARD,2,10/17/2011 15:27,0.83,NaN,United Kingdom


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [110]:
dateliste = 'JJ/MM/AAAA'.split()
dateliste[0]

'JJ/MM/AAAA'

In [111]:
# Formatage de la colonne "InvoiceDate" pour supprimer l'heure et ne garder que JJ/MM/AAAA

# on transforme les données en liste
df['InvoiceDate'] = df['InvoiceDate'].apply(str.split) 

# on ne garde que le 1er élément de la liste indice 0
df['InvoiceDate'] = df['InvoiceDate'].apply(lambda x : x[0]) 

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate']


0        2010-12-01
1        2010-12-01
2        2010-12-01
3        2010-12-01
4        2010-12-01
            ...    
541904   2011-12-09
541905   2011-12-09
541906   2011-12-09
541907   2011-12-09
541908   2011-12-09
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns]

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [113]:
# Création d'une colonne "TotalCA"

df['TotalCA'] = round(df['Quantity'] * df['UnitPrice'],2)

df['TotalCA'] = df['TotalCA'].apply(lambda x: 0 if x <= 0 else x)

df['TotalCA'] = df['TotalCA'].apply(lambda x: str(x).replace(".", ","))


In [114]:
# Remplir les descriptions nulles avec celles en majuscules si StockCode correspond

# for index, row in df.iterrows():
#     if pd.isna(row['Description']):  # Vérifie si la description est NaN
#         stock_code = row['StockCode']  # Stocke le StockCode correspondant
#         # Cherche une description en majuscule dans le même StockCode
#         matching_rows = df[
#             (df['StockCode'] == stock_code) & 
#             (df['Description'].str.isupper() == True)
#         ]
#         if not matching_rows.empty:  # Si une ligne correspondante est trouvée
#         # if len(matching_rows) > 0: # Alternative
#             df.at[index, 'Description'] = matching_rows['Description'].iloc[0]


In [115]:
# Fonction pour calculer le % de majuscules
def pourcent_maj(texte):
    maj_compte = sum(1 for caract in texte if caract.isupper())
    total = sum(1 for caract in texte if caract.isalpha())
    if total != 0:
        return maj_compte / total * 100
    else : 
        return 0

In [116]:
# Dico de référence CODE / DESCRIPTION

dico_ref = {}

for stock_code, groupe in df.groupby('StockCode'):
    descriptions = groupe['Description'].unique()
    plus_gd_pourcent = max(descriptions, key=lambda x : pourcent_maj(str(x)))
    dico_ref[stock_code] = plus_gd_pourcent


def application(row):
    if row['StockCode'] in dico_ref.keys():
        return dico_ref[row['StockCode']]
    else:
        return row['Description']
    

df['Description'] = df.apply(application, axis=1)

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  541797 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
 8   TotalCA      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 37.2+ MB


In [118]:

df['UnitPrice'] = df['UnitPrice'].apply(lambda x: str(x).replace(".", ","))
df['UnitPrice']


0         2,55
1         3,39
2         2,75
3         3,39
4         3,39
          ... 
541904    0,85
541905     2,1
541906    4,15
541907    4,15
541908    4,95
Name: UnitPrice, Length: 541909, dtype: object

In [119]:
df['CustomerID'] = df['CustomerID'].fillna(0)
df['CustomerID'] = df['CustomerID'].astype(int)
df['CustomerID']

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 541909, dtype: int32

In [120]:
# Export en CSV
df.to_csv("data_clean.csv")